In [ ]:
! pip install gensim==3.8.3

In [ ]:
from gensim.summarization.summarizer import summarize
import pandas as pd
import numpy as np
from datetime import datetime
import re

### 함수

In [ ]:
# 정제
def cleaning(text) :
    text = str(text)

    pattern = r"▲ 에코프로 포항사업장\.[^\|]+|에코프로 제공\|+|에코프로 자료사진\|+|에코프로 홈페이지\|+|이메일 : jebo@cbs.co.kr|카카오톡 : @노컷뉴스사이트 : https://url.kr/b71afn|SBS Biz는 여러분의 제보를 기다리고 있습니다.홈페이지 = https://url.kr/9pghjn|.■ 제보하기▷ 전화 : 02-781-1234, 4444▷ 이메일 : kbs1234@kbs.co.kr▷ 카카오톡 : 'KBS제보' 검색, 채널 추가▷ 네이버, 유튜브에서 KBS뉴스를 구독해주세요!|MBC 뉴스는 24시간 여러분의 제보를 기다립니다. ▷ 전화 02-784-4000▷ 이메일 mbcjebo@mbc.co.kr▷ 카카오톡 @mbc제보|/사진제공=네이버증권|/사진=|한경DB|당신의 제보가 뉴스로 만들어집니다"
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = r'[\n\t]'    # 개행 문자, 탭 문자 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    text = text.replace('↓', '하락')
    text = text.replace('↑', '상승')
    text = text.replace('→', '에서')
    text = text.replace('%', '퍼센트')
    text = text.replace('…', '.')
    text = text.replace("2차전지",'이차전지')

    pattern = r'\([^)]{1,50}\)|\[[^\]]{1,50}\]|<[^>]{1,50}>' # 대개 기자 이름, 사진 설명 등
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    #pattern = r'(\d{4}\.\d{2}\.\d{2}\.)|(\d{4}\.\d{2}\.\d{2})' # 날짜 제거
    #text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = r'[^\uAC00-\uD7A3 0-9a-zA-Z\s.,]'  # 한글, 숫자, 영어, 온점, 쉼 제외 모두 제거 (수치 데이터가 많아서 온점 제외 X)
#   pattern = r'[^가-힣ㄱ-ㅎㅏ-ㅣ]'               # 한글 제외 모두 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    pattern = r'\b\w{2,5}\s?(?:기자|기자입니다)\b'   # 기자 이름 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)

    #pattern = '|'.join(sm['언론사'])
    #text = re.sub(pattern = pattern, repl = ' ', string = text)

    #text= re.sub(r'[^0-9.%]', '', string = text)  # 수치 제외 온점 제거

    text = re.sub(r'\s+', ' ', text) # 연속 공백 제거

    return text

In [ ]:
def news_summarize(df) :
    error_indices = []

    for i in range(len(df)) :
        text  = df.loc[i,'본문']

        try :
            summary = summarize(text, word_count = 100)
            df.loc[i, '요약'] = summary

            if not summary.strip() :
                df.loc[i, '요약'] = fail_summarize(df, text, i)

        except ValueError as e:
            df.loc[i, '요약'] = fail_summarize(df, text, i) #여러 개의 문장을 한 문장으로 인식 -> fail_summarize()로 해결

    return df

In [ ]:
def fail_summarize(df, text, i) :
    tmp = text.replace('다.','다.\n')
    text_list = tmp.split("\n")

    if 1 <= len(text_list) <= 9 :
        if len(text_list) == 1 :  # 사진 설명과 같이 의미없는 내용들이 있어서 제목으로 감성분석하게 제목 추가함
            title = df.loc[i,'제목']
            text += title
        return text

    elif len(text_list) >= 10 : # 10문장을 초과해야 요약문을 넣더라
        if len(text_list) == 10 : # 딱 10문장인 기사는 제목을 추가해서 11문장으로 만들어주고 요약 활성화
            title = df.loc[i,'제목']
            text_list.append(title)
        text = "\n".join(text_list)
        summary = summarize(text, word_count = 100)  # word_count 값에 따라 빈 요약문이 return 되는 경우도 있음/ ratio를 설정하면 일부 문장이 너무 길어져 요약 의미가 없었음
        summary = re.sub(pattern = r'[\n]', repl = ' ', string = summary)

        return summary

### SM 요약

In [ ]:
# 데이터 불러오기
sm = pd.read_csv("../data/뉴스_sm.csv")
sm.head()

,언론사,제목,작성일시,본문,링크
0,한국경제,"""K팝 가수 깨알 정보까지 한눈에 제공""",2023-02-01,김홍기 스페이스오디티 대표팬덤 플랫폼 '블립' 운영\n\n\n\n“K팝 스타에 대해...,https://n.news.naver.com/mnews/article/015/000...
1,한국경제,"""올해 신인그룹 속속 데뷔""…기대 커진 엔터주 동반강세",2023-02-01,"하이브, 걸그룹 '뉴진스' 돌풍방탄소년단 공백 거뜬히 메워JYP엔터, 모두 4팀 출...",https://n.news.naver.com/mnews/article/015/000...
2,조선비즈,뉴진스는 좋지만 배당은 아쉽다...시총 7.7조 ‘하이브’ 주주환원 방안에는 침묵,2023-02-02,SM엔터테인먼트 지난해 첫 배당 실시 그룹 방탄소년단(BTS) 공백에도...,https://n.news.naver.com/mnews/article/366/000...
3,더스쿠프,SM엔터가 내놓고 SM엔터가 흔드는 묘한 M&A,2023-02-02,"매물로 나온지 한참 흐른 SM엔터군불만 뗄 뿐 거래 이어지지 않아 카카오엔터, CJ...",https://n.news.naver.com/mnews/article/665/000...
4,더팩트,"[엔터Biz-D] SM엔터, 'SM 3.0' 기대감에 사상 첫 9만 원 돌파",2023-02-03,전 거래일 대비 2.13%↑\n\n\n\n3일 SM엔터테인먼트는 전 거래일 대비 2...,https://n.news.naver.com/mnews/article/629/000...


In [ ]:
sm.isnull().sum()

언론사     0
제목      0
작성일시    0
본문      1
링크      0
dtype: int64

In [ ]:
# 결측치 제거
sm = sm.dropna(axis=0)

# 중복값 제거
sm = sm.drop_duplicates(['링크'],keep='first')

# 제목만 다르고, 본문 내용은 같은 기사 제거 (같은 날 작성)
sm = sm.drop_duplicates(['본문','작성일시'], keep = 'first')

# 헤드라인 기사
sm = sm[~sm['제목'].str.contains('헤드라인')]
sm = sm[~sm['본문'].str.contains('헤드라인')]
sm = sm[~sm['제목'].str.contains('전국 주요 신문 톱뉴스')]

# 인덱스 재정렬
sm = sm.reset_index(drop=True)

In [ ]:
sm.head()

,언론사,제목,작성일시,본문,링크
0,한국경제,K팝 가수 깨알 정보까지 한눈에 제공,2023-02-01,김홍기 스페이스오디티 대표팬덤 플랫폼 블립 운영 K팝 스타에 대해 미주알고주알 깨알...,https://n.news.naver.com/mnews/article/015/000...
1,한국경제,올해 신인그룹 속속 데뷔 .기대 커진 엔터주 동반강세,2023-02-01,"하이브, 걸그룹 뉴진스 돌풍방탄소년단 공백 거뜬히 메워JYP엔터, 모두 4팀 출격 ...",https://n.news.naver.com/mnews/article/015/000...
2,조선비즈,뉴진스는 좋지만 배당은 아쉽다...시총 7.7조 하이브 주주환원 방안에는 침묵,2023-02-02,SM엔터테인먼트 지난해 첫 배당 실시 그룹 방탄소년단 공백에도 하이브의 주가 상승세...,https://n.news.naver.com/mnews/article/366/000...
3,더스쿠프,SM엔터가 내놓고 SM엔터가 흔드는 묘한 M A,2023-02-02,"매물로 나온지 한참 흐른 SM엔터군불만 뗄 뿐 거래 이어지지 않아 카카오엔터, CJ...",https://n.news.naver.com/mnews/article/665/000...
4,더팩트,"SM엔터, SM 3.0 기대감에 사상 첫 9만 원 돌파",2023-02-03,전 거래일 대비 2.13퍼센트상승 3일 SM엔터테인먼트는 전 거래일 대비 2.13퍼...,https://n.news.naver.com/mnews/article/629/000...


In [ ]:
sm['제목'] = sm['제목'].apply(cleaning)
sm['본문'] = sm['본문'].apply(cleaning)

In [ ]:
# 전처리과정에서 빈 문자열인 데이터 제거
sm = sm[sm['제목'] != ' ']

# 인덱스 재정렬
sm = sm.reset_index(drop=True)

In [ ]:
sm.tail()

,언론사,제목,작성일시,본문,링크
2181,SBS Biz,뒤숭숭한 카카오...항의서한 받은 김범수 창업주,2023-07-27,"김범수 카카오 미래이니셔티브 센터장.카카오가 안팎으로 뒤숭숭하면서, 입지가 흔들리...",https://n.news.naver.com/mnews/article/374/000...
2182,이코노미스트,17만원에서 4만원대.연 이은 악재에 바람 잘 날 없는 카카오,2023-07-28,시세조종 의혹 계열사 실적 부진 등 악재수사 결과 따라 주가 추가 하락 불가피2분기...,https://n.news.naver.com/mnews/article/243/000...
2183,중앙SUNDAY,그가 없었다면 지금의 K팝도 없다,2023-07-29,이수만 연구 이수만 연구 전영선 배정원 황지영 박건 지음 커뮤니케이션북스 현재 한...,https://n.news.naver.com/mnews/article/353/000...
2184,매경이코노미,"실적과 주가는 반 토막, 직원들은 단체행동.사면초가 카카오",2023-07-31,네이버와 더불어 한국 IT 산업의 양강으로 꼽히는 카카오가 흔들린다. 실적과 주가는...,https://n.news.naver.com/mnews/article/024/000...
2185,서울경제,"SM엔터 7인조 신인 보이그룹 라이즈, 9월 데뷔",2023-07-31,NCT 이후 7년만 보이그룹 데뷔 SM엔터테인먼트 신인 보이그룹 라이즈 로고. 사진...,https://n.news.naver.com/mnews/article/011/000...


In [ ]:
sm = news_summarize(sm)

In [ ]:
sm = sm.drop(['언론사','링크'],axis=1)

In [ ]:
sm.isnull().sum()

제목      0
작성일시    0
본문      0
요약      0
dtype: int64

In [ ]:
sm[sm['요약']=='']

,제목,작성일시,본문,요약


In [ ]:
sm.to_csv('../data/news_sm_요약추가.csv', index=None)

### 에코프로

In [ ]:
batt = pd.read_csv("../data/뉴스_에코프로.csv")
batt.head()

,언론사,제목,작성일시,본문,링크
0,서울경제,"[데이터로 보는 증시]에코프로비엠, 기관·외국인 코스닥 순매수 1위(2월 1일-최종치)",2023-02-01,"다믈멀티미디어, 개인 순매수 1위[서울경제]",https://n.news.naver.com/mnews/article/011/000...
1,매일경제,반도체만 바라보는 '천수답' 韓수출 … 하반기엔 보릿고개 넘을까,2023-02-01,반도체수출액 108억→60억弗D램값도 역대최저 '호재 실종'對中수출액 31% 쪼그라...,https://n.news.naver.com/mnews/article/009/000...
2,머니투데이,내일 새벽 나올 FOMC 회의 결과…파월은 어떤 입장 내 놓을까,2023-02-01,\t\t\t[내일의 전략]\n\n\n\n제롬 파월 미국 연방준비제도 이사회 의장/A...,https://n.news.naver.com/mnews/article/008/000...
3,아이뉴스24,"코스피, 외인 매수에 반등…2450선 턱밑",2023-02-01,코스닥도 1.4%↑코스피지수가 외국인 매수세에 상승 마감했다.\n\n\n\n코스피가...,https://n.news.naver.com/mnews/article/031/000...
4,연합뉴스,"코스피, FOMC 앞두고 1% 상승 마감…외국인 순매수 전환(종합)",2023-02-01,"전날 하락한 삼성전자·SK하이닉스 반등\n\n\n\n코스피, 2,440대로 올라서 ...",https://n.news.naver.com/mnews/article/001/001...


In [ ]:
# 결측치 제거
batt = batt.dropna(axis=0)

# 중복값 제거
batt = batt.drop_duplicates(['링크'], keep = 'first')

# 제목만 다르고, 본문 내용은 같은 기사 제거 (같은 날 작성)
batt = batt.drop_duplicates(['본문'], keep = 'first')

# 본문에 사진 설명만 또는 아무 내용이 없는 기사 + 무관한 기사 제거
batt = batt[~batt['본문'].str.contains('전지훈련')]

batt = batt[~batt['제목'].str.contains('헤드라인')]
batt = batt[~batt['본문'].str.contains('헤드라인')]

batt = batt[batt['본문'].str.contains('2차 전지|2차전지|에코프로|이차전지')]

# 인덱스 재정렬
batt = batt.reset_index(drop=True)

,언론사,제목,작성일시,본문,링크
0,매일경제,반도체만 바라보는 '천수답' 韓수출 … 하반기엔 보릿고개 넘을까,2023-02-01,반도체수출액 108억→60억弗D램값도 역대최저 '호재 실종'對中수출액 31% 쪼그라...,https://n.news.naver.com/mnews/article/009/000...
1,머니투데이,내일 새벽 나올 FOMC 회의 결과…파월은 어떤 입장 내 놓을까,2023-02-01,\t\t\t[내일의 전략]\n\n\n\n제롬 파월 미국 연방준비제도 이사회 의장/A...,https://n.news.naver.com/mnews/article/008/000...
2,아이뉴스24,"코스피, 외인 매수에 반등…2450선 턱밑",2023-02-01,코스닥도 1.4%↑코스피지수가 외국인 매수세에 상승 마감했다.\n\n\n\n코스피가...,https://n.news.naver.com/mnews/article/031/000...
3,연합뉴스,"코스피, FOMC 앞두고 1% 상승 마감…외국인 순매수 전환(종합)",2023-02-01,"전날 하락한 삼성전자·SK하이닉스 반등\n\n\n\n코스피, 2,440대로 올라서 ...",https://n.news.naver.com/mnews/article/001/001...
4,서울경제,삼성 2.3조 SK 0.4조…날아간 반도체 稅혜택,2023-02-01,■ 법인세 '최저한세'에 발목잡힌 투자 세액공제최대 25% 공제율 적용때 실효세율삼...,https://n.news.naver.com/mnews/article/011/000...
...,...,...,...,...,...
19452,데일리안,대신證 “2차전지株 반등은 데드 캣 바운스…후폭풍 경계”,2023-07-31,외인 수급 주도권 가져갈지 관건\n\n\n\nⓒ대신증권[데일리안 = 황인욱 기자] ...,https://n.news.naver.com/mnews/article/119/000...
19453,더팩트,"[시황] 코스피, 美훈풍 개인 '사자' 상승 마감…2차전지 강세 여전",2023-07-31,코스닥 시가총액 상위 10개 종목 모두 빨간불\n\n\n\n31일 코스피는 전날보다...,https://n.news.naver.com/mnews/article/629/000...
19454,매일경제,북미 인프라 수혜株 고공행진,2023-07-31,실적호조 효성重 상한가LS일렉 변압기 매출호조공급망에서 중국 의존도를 줄이고 공급처...,https://n.news.naver.com/mnews/article/009/000...
19455,아시아경제,"웰바이오텍, 리튬 원광 1차분 선적…""고부가가치 사업 확대""",2023-07-31,\t\t\t웰바이오텍이 2차전지의 필수재인 리튬 사업을 본격적으로 시작한다. 짐바브...,https://n.news.naver.com/mnews/article/277/000...


In [ ]:
batt['제목'] = batt['제목'].apply(cleaning)
batt['본문'] = batt['본문'].apply(cleaning)

In [ ]:
# 전처리과정에서 빈 문자열인 데이터 제거  ([] 안 내용 제거 -> 주로 헤드라인 기사 도는 사진 설명만 있는 기사)
batt = batt[batt['제목'] != ' ']
batt = batt[batt['본문'] != ' ']

# 인덱스 재정렬
batt = batt.reset_index(drop=True)

In [ ]:
batt.tail()

,언론사,제목,작성일시,본문,링크
19405,데일리안,대신 이차전지 반등은 데드 캣 바운스.후폭풍 경계,2023-07-31,외인 수급 주도권 가져갈지 관건 대신증권 대신증권은 31일 이차전지 관련주의 급락 ...,https://n.news.naver.com/mnews/article/119/000...
19406,더팩트,"코스피, 훈풍 개인 사자 상승 마감.이차전지 강세 여전",2023-07-31,코스닥 시가총액 상위 10개 종목 모두 빨간불 31일 코스피는 전날보다 24.26포...,https://n.news.naver.com/mnews/article/629/000...
19407,매일경제,북미 인프라 수혜 고공행진,2023-07-31,실적호조 효성 상한가LS일렉 변압기 매출호조공급망에서 중국 의존도를 줄이고 공급처를...,https://n.news.naver.com/mnews/article/009/000...
19408,아시아경제,"웰바이오텍, 리튬 원광 1차분 선적. 고부가가치 사업 확대",2023-07-31,웰바이오텍이 이차전지의 필수재인 리튬 사업을 본격적으로 시작한다. 짐바브웨에서 채...,https://n.news.naver.com/mnews/article/277/000...
19409,파이낸셜뉴스,여전히 불타는 이차전지 투심...금양 에코프로 상승랠리,2023-07-31,이차전지 종목의 투자심리가 꺼지지 않고 있다. 대장주들의 급등세가 이어지며 달아오...,https://n.news.naver.com/mnews/article/014/000...


In [ ]:
batt = batt.drop(['언론사','링크'],axis=1)

In [ ]:
batt = news_summarize(batt)

In [ ]:
batt.isnull().sum()

제목      0
작성일시    0
본문      0
요약      0
dtype: int64

In [ ]:
# batt[batt['요약'] ==''] word_count에 따라 달라지는 거 보여주기
text = batt.loc[11139,'본문']
tmp = text.replace('다.','다.\n')
text_list = tmp.split('\n')
text = '\n'.join(text_list)
summary_100 = summarize(text,word_count=100)
summary_150 = summarize(text,word_count=150)
print('word_count=100 : ', summary_100)
print('word_count=150 : ', summary_150)

word_count=100 :  
word_count=150 :  출연 염승환 이베스트 투자증권 이사파월, 다음 달 기준금리 동결 시사 배경은 염승환 연준이 나서서 추가적으로 금리 인상까지 할 필요는 없어졌다는 해석으로 시장에서 받아들이고 있어 한은, 기준금리 동결 가능성 전망은 염승환 파월 의장 발언 나왔기 때문에 무리해서 금리 올릴 이유 더 없어져 코스피 2,550대 출발 증시 전망은 염승환 종가 기준으로 올해 최고가 2575포인트였는데, 돌파가능성 높아져 염승환 지금은 2차 전지도 강한데 여기에 반도체 같이 가고 있는 상황 염승환 반도체, 조선, 건설은 흔히 경기 민감주인데 이 업종들이 전체적으로 다 오른다는 건 시장을 긍정적으로 본다는 시그널 반도체 제조 시설 일본행 이유는 염승환 대만과 중국 쪽에서는 조금씩 힘을 빼겠다는 그런 입장인 것 같아 염승환 일본 같은 경우 반도체 산업, 미국 규제 심해지면서 사실상 한국과 대만에 완전히 밀렸는데, 다시 한번 부흥하려는 것 염승환 우리 한국 입장에서도 중국 비중을 줄일 수밖에 없는 상황 중국, 미국 반도체 기업 마이크론 첫 직접 제재 이유는 염승환 메모리 반도체 만드는 회사가 삼성, SK하이닉스, 마이크론 3곳인데 마이크론 쓰지 말라는 것 염승환 중국은 반도체 제조 역량 떨어지기 때문에 한국 반도체 안 쓰면 사실상 마이크론 대체할 방법없어 삼성 반도체 시장 전략은 염승환 삼성의 최신 기술이 글로벌 탈공급망 시대에 결국 빛을 발할 것 연기금, 조선주 8백억 대 순매수 이유는 염승환 선박 발주 늘어날 때 한국이 가장 수혜를 받지 않을까 하는 기대감으로 주가 강세 보여 앵커 경제 이야기 나누어 보겠습니다.


In [ ]:
batt = batt[batt['요약'] !='']  # 2개 행은 word_count=100으로 설정하면 빈 요약문이 생성됨 + 에코프로 관련 내용이 아니거나 헤드라인 기사
batt = batt.reset_index(drop=True)

In [ ]:
batt.to_csv('../data/news_에코프로_요약추가.csv', index=None)